In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time

driver = webdriver.Chrome()

driver.get("http://jhsjk.people.cn/")

# 点击“讲话”按钮
jhButton = driver.find_element(By.XPATH, '/html/body/div/div[3]/a[1]')
jhButton.click()

# Switch to the new tab
driver.switch_to.window(driver.window_handles[-1])

# Switch back to the original tab if needed
# driver.switch_to.window(driver.window_handles[0])

In [57]:
# Function to get URL from list items, 10 list items per page
def get_urls():
    urls = []
    for i in range(1, 11):
        xpath = f"/html/body/div[6]/div[1]/ul/li[{i}]/a"
        try:
            element = driver.find_element(By.XPATH, xpath)
            url = element.get_attribute("href")
            urls.append(url)
        except:
            break
    return urls

In [58]:
urls = []
# get_urls() on 1st page
urls.extend(get_urls())

# 2nd page
driver.find_element(By.XPATH, '/html/body/div[6]/div[1]/div/span[2]').click()
time.sleep(0.5)
urls.extend(get_urls())

# 3rd page
driver.find_element(By.XPATH, '/html/body/div[6]/div[1]/div/span[3]').click()
time.sleep(0.5)
urls.extend(get_urls())

# All pages afterwards
while True:
    try:
        driver.find_element(By.XPATH, '/html/body/div[6]/div[1]/div/span[4]').click()
        time.sleep(0.5)
        urls.extend(get_urls())
    except:
        break

urls = list(set(urls))

In [4]:
# Save scrapped URLs to file

with open('jhsjk_urls.txt', 'w') as file:
    for url in urls:
        file.write(url + '\n')

In [2]:
# Read URLs from file
urls = []
with open('jhsjk_urls.txt', 'r') as file:
    for line in file:
        url = line.strip()
        urls.append(url)

In [1]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm


directory = r'text_output/jhsjk'
if not os.path.exists(directory):
    os.makedirs(directory)


def save_text(urls):
    errUrls = []
    for url in tqdm(urls):
        try:
            page = requests.get(url,
                                headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36"}
                                )
            soup = BeautifulSoup(page.text, 'html.parser')

            title = soup.find('h1').get_text(strip=True).replace('\n', ' ')
            
            headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
            paragraphs = soup.find_all('p')
            div_text = soup.find('div', class_='d2txt_1 clearfix')

            filtered_text = [tag.get_text(strip=True) for tag in [div_text] + headers + [paragraphs[0]]]

            file_path = f'{directory}/{title}.txt'
            i = 1
            while os.path.exists(file_path):
                modified_title = f'{title}_{i}'
                file_path = f'{directory}/{modified_title}.txt'
                i += 1
            
            with open(file_path, 'w') as file:
                for i, text in enumerate(filtered_text):
                    if i == len(filtered_text) - 1:
                        file.write(text)
                    else:
                        file.write(text + '\n')

        except:
            print(f"Error processing {url}")
            errUrls.append(url)
    return errUrls


In [3]:
# Will attempt to save once more the failed URLs
errURLs = save_text(urls)
save_text(errURLs)

  0%|          | 0/800 [00:00<?, ?it/s]

100%|██████████| 800/800 [09:06<00:00,  1.46it/s]
0it [00:00, ?it/s]


[]